We have done text classification using count vectorizer then tfidf vectorizer then word2vec

and now we will try with sentence bert 

In [1]:
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.5 MB/s eta 0:00:00


In [2]:

import pandas as pd
import numpy as np
np.random.seed(2022)
     

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
df = pd.read_csv("/content/toxic_comment (1).csv")

In [7]:
df.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,00562e78e0b34102,"Yeah, let's merge the content. (Not sure if De...",0,0,0,0,0,0
1,1,00ef397f0d9e599a,REDIRECT Talk:A Voice Within (album),0,0,0,0,0,0
2,2,00f0a2af234e1950,The discussion of when there first were outdoo...,0,0,0,0,0,0
3,3,00b984b355cc9754,I'm focussing on doing science at the moment (...,0,0,0,0,0,0
4,4,0006f16e4e9f292e,Before you start throwing accusations and warn...,0,0,0,0,0,0


In [8]:
df = df[["comment_text","toxic"]]

In [9]:
df.head()

,comment_text,toxic
0,"Yeah, let's merge the content. (Not sure if De...",0
1,REDIRECT Talk:A Voice Within (album),0
2,The discussion of when there first were outdoo...,0
3,I'm focussing on doing science at the moment (...,0
4,Before you start throwing accusations and warn...,0


In [10]:
import spacy
import string
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


{"'d", 'sometimes', 'twenty', 'becoming', 'anything', 'ourselves', 'over', '‘s', 'anyone', 'everything', 'several', 'without', 'until', 'itself', 'in', 'amongst', 'often', 'thereafter', 'should', 'whatever', 'they', 'ten', 'nowhere', 'can', 'nine', 'along', 'at', "'re", 'mostly', 'other', 'upon', 'whoever', 'used', 'around', 'so', 'hereby', 'while', 'but', 'ever', 'whereupon', 'myself', 'an', 'amount', 'part', 'my', 'beyond', 'take', 'noone', 'full', 'than', 'these', 'otherwise', 'wherein', 'this', 'back', '’ve', 'enough', 'yourself', 'been', 'thereby', 'first', 'did', 'yours', 'very', 'two', 'our', 'almost', 'get', 'ca', 'although', 'must', "'s", 'eight', 'that', 'there', 'became', 'whereas', 'might', 'fifteen', 'seems', 'moreover', 'rather', 'more', 'four', 'using', 'its', 'once', 'neither', 'same', 'why', 'towards', 'quite', 'whole', 'therefore', 'have', 'to', 'about', 'therein', 'whence', 'again', 'had', 'seemed', 'always', 'herself', 'n‘t', "'ve", 'thence', 'would', 'ours', '’d', 

In [11]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
# Creating our tokenizer function..  convert sentence to spacy object on which we can iterate then we strip each 
# word and find its meaningful root words then we make list of words which are not punctuation and 
#which are not stop words then we join them . thats it
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    sentence = " ".join(mytokens)
    return sentence

In [13]:

spacy_tokenizer("I am Eating")
     

'eat'

In [15]:
df['tokenize'] = df['comment_text'].apply(spacy_tokenizer)

In [18]:
df.head()

,comment_text,toxic,tokenize
0,"Yeah, let's merge the content. (Not sure if De...",0,yeah let merge content sure devil canyon type ...
1,REDIRECT Talk:A Voice Within (album),0,redirect talk voice album
2,The discussion of when there first were outdoo...,0,discussion outdoor tree electric light strange...
3,I'm focussing on doing science at the moment (...,0,focusse science moment lead revolution
4,Before you start throwing accusations and warn...,0,start throw accusation warning let review edit...


In [19]:
df['embeddings'] = df['tokenize'].apply(model.encode)

In [20]:
df.head()

,comment_text,toxic,tokenize,embeddings
0,"Yeah, let's merge the content. (Not sure if De...",0,yeah let merge content sure devil canyon type ...,"[-0.03033301, -0.08857519, -0.10544955, 0.0085..."
1,REDIRECT Talk:A Voice Within (album),0,redirect talk voice album,"[0.017644601, -0.0227581, -0.013780398, -0.015..."
2,The discussion of when there first were outdoo...,0,discussion outdoor tree electric light strange...,"[-0.008159327, 0.13027689, 0.049433626, 0.0654..."
3,I'm focussing on doing science at the moment (...,0,focusse science moment lead revolution,"[0.0028992058, 0.012477529, 0.0043988074, 0.01..."
4,Before you start throwing accusations and warn...,0,start throw accusation warning let review edit...,"[0.038920697, 0.04419447, -0.0037439559, 0.035..."


In [21]:
X = df['embeddings'].to_list()
y = df['toxic'].values
#we do to_list because we dont want pandas series so to_list or also we can do 
# .values() it will also works same

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)


In [23]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [24]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.88
Logistic Regression Precision: 0.8888888888888888
Logistic Regression Recall: 0.8


What we notice sentence embedding works little better then word embeddings and word embedding works little better than tfidf and count vectorizer . 

Count Vectorizer contain least information and Tfidf contain little more info . but word2vec contain more inforamtion of that word than any other vetorizer



